In [ ]:
# ============================================
# SISTEMA DE DETECCIÓN DE ENLACES SPAM
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
print("Librerias Cargadas")

In [ ]:
# Configuración visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

Cargar datos

In [ ]:
url = 'https://breathecode.herokuapp.com/asset/internal-link?id=932&path=url_spam.csv'
df = pd.read_csv(url)
print("Dataset Cargado")

In [ ]:
print("\n Información general del dataset:")
print(df.info())

In [ ]:
print("\n Primeras filas:")
print(df.head(10))

In [ ]:
print("\n Estadísticas descriptivas:")
print(df.describe())

In [ ]:
# Identificar columnas
columna_url = df.columns[0]
columna_spam = df.columns[1]

print(f"\nColumnas identificadas:")
print(f"  - URLs: '{columna_url}'")
print(f"  - Etiquetas: '{columna_spam}'")

In [ ]:
# Distribución de clases
print("\n Distribución de clases:")
print(df[columna_spam].value_counts())
print(f"\nPrcentaje de spam: {df[columna_spam].mean()*100:.2f}%")

In [ ]:
# Visualización 1: Distribución de clases
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Gráfico de barras
df[columna_spam].value_counts().plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Distribución: Spam vs No Spam')
axes[0].set_xlabel('Clase (0=No Spam, 1=Spam)')
axes[0].set_ylabel('Cantidad')
axes[0].set_xticklabels(['No Spam', 'Spam'], rotation=0)

# Análisis de longitud de URLs
df['longitud_url'] = df[columna_url].str.len()
df['longitud_url'].hist(bins=50, ax=axes[1], edgecolor='black', alpha=0.7)
axes[1].set_title('Distribución de Longitud de URLs')
axes[1].set_xlabel('Longitud (caracteres)')
axes[1].set_ylabel('Frecuencia')

# Longitud por clase
df.boxplot(column='longitud_url', by=columna_spam, ax=axes[2])
axes[2].set_title('Longitud de URL por Clase')
axes[2].set_xlabel('Clase (0=No Spam, 1=Spam)')
axes[2].set_ylabel('Longitud')

plt.tight_layout()
plt.show()

In [ ]:
print("\n Estadísticas de longitud por clase:")
print(df.groupby(columna_spam)['longitud_url'].describe())

In [ ]:
# Análisis de caracteres especiales
df['num_puntos'] = df[columna_url].str.count(r'\.')
df['num_guiones'] = df[columna_url].str.count(r'-')
df['num_barras'] = df[columna_url].str.count(r'/')
df['num_digitos'] = df[columna_url].str.count(r'\d')

print("\n Características promedio por clase:")
features_eda = ['longitud_url', 'num_puntos', 'num_guiones', 'num_barras', 'num_digitos']
print(df.groupby(columna_spam)[features_eda].mean())

PREPROCESAMIENTO

In [ ]:
def limpiar_url(url):
    """
    Limpia y tokeniza una URL para el modelo
    """
    if pd.isna(url):
        return ''
    
    # Convertir a minúsculas
    url = str(url).lower()
    # Quitar protocolo y www
    url = url.replace('http://', '').replace('https://', '').replace('www.', '')
    
    # Separar por caracteres especiales (convertirlos en espacios)
    url = re.sub(r'[/\-_\.?=&@:+%#\(\)\[\]]', ' ', url)
    
    # Quitar espacios múltiples
    url = ' '.join(url.split())
    
    return url

Aplicar limpieza

In [ ]:
df['url_limpia'] = df[columna_url].apply(limpiar_url)

print("\n Ejemplos de URLs preprocesadas:")
for i in range(5):
    print(f"\nOriginal: {df[columna_url].iloc[i]}")
    print(f"Limpia:   {df['url_limpia'].iloc[i]}")
    print(f"Spam     {df[columna_spam].iloc[i]}")

In [ ]:
# Preparar X e y
X = df['url_limpia']
y = df[columna_spam]

# División train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\nDivisión de datos:")
print(f"  Train: {len(X_train)} URLs ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Test:  {len(X_test)} URLs ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n  Distribución en Train:")
print(f"    No Spam: {(y_train==0).sum()}")
print(f"    Spam:    {(y_train==1).sum()}")

Vectorización con TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(
    max_features=1500,   # Top 1500 términos más importantes
    ngram_range=(1, 2),  # Unigramas y bigramas
    min_df=2,            # Ignorar términos que aparecen en menos de 2 documentos
    max_df=0.95          # Ignorar términos muy frecuentes
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(f"\n Vectorización completada:")
print(f"  Train shape: {X_train_vec.shape}")
print(f"  Test shape:  {X_test_vec.shape}")

SVM CON PARÁMETROS POR DEFECTO

In [ ]:
# Entrenar SVM 
svm_base = SVC(kernel='linear', random_state=42)
print("\n⏳ Entrenando SVM...")
svm_base.fit(X_train_vec, y_train)

In [ ]:
# Predicciones
y_pred_train = svm_base.predict(X_train_vec)
y_pred_test = svm_base.predict(X_test_vec)

In [ ]:
# Métricas
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)


In [ ]:
print(f"\nResultados SVM base:")
print(f"  Accuracy Train: {acc_train:.4f}")
print(f"  Accuracy Test:  {acc_test:.4f}")

In [ ]:
print("\n Reporte de clasificación (Test):")
print(classification_report(y_test, y_pred_test, target_names=['No Spam', 'Spam']))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Spam', 'Spam'],
            yticklabels=['No Spam', 'Spam'])
plt.title('Matriz de Confusión - SVM Base')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()


Analisis de Resultados:
1. Verdaderos Negativos (448): URLs que realmente no eran spam y tu modelo también las clasificó como “No Spam”. 
2. Verdaderos Positivos (110): URLs correctamente identificadas como spam.
3. Falsos Positivos (13): URLs legítimas que fueron clasificadas como spam por error.
4. Falsos Negativos (29): URLs spam que el modelo no detectó.

OPTIMIZACIÓN CON GRID SEARCH

In [ ]:
# Definir grid de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],           # Regularización
    'kernel': ['linear', 'rbf'],       # Tipo de kernel
    'gamma': ['scale', 'auto', 0.1, 1] # Para kernel rbf
}

print("\n Buscando mejores hiperparámetros...")
print(f"Grid de búsqueda: {param_grid}")

In [ ]:
# Grid Search con validación cruzada
grid_search = GridSearchCV(
    SVC(random_state=42),
    param_grid,
    cv=5,                    # 5-fold cross validation
    scoring='accuracy',
    n_jobs=-1,              # Usar todos los cores
    verbose=1
)

grid_search.fit(X_train_vec, y_train)

print(f"\nMejores parámetros encontrados:")
print(grid_search.best_params_)
print(f"\nMejor score en validación cruzada: {grid_search.best_score_:.4f}")

In [ ]:
# Mejor modelo
svm_optimizado = grid_search.best_estimator_


In [ ]:
# Evaluar modelo optimizado
y_pred_train_opt = svm_optimizado.predict(X_train_vec)
y_pred_test_opt = svm_optimizado.predict(X_test_vec)

acc_train_opt = accuracy_score(y_train, y_pred_train_opt)
acc_test_opt = accuracy_score(y_test, y_pred_test_opt)

print(f"\n Resultados SVM optimizado:")
print(f"  Accuracy Train: {acc_train_opt:.4f}")
print(f"  Accuracy Test:  {acc_test_opt:.4f}")

print("\n Reporte de clasificación (Test - Optimizado):")
print(classification_report(y_test, y_pred_test_opt, target_names=['No Spam', 'Spam']))

In [ ]:
# Matriz de confusión optimizada
cm_opt = confusion_matrix(y_test, y_pred_test_opt)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_opt, annot=True, fmt='d', cmap='Greens',
            xticklabels=['No Spam', 'Spam'],
            yticklabels=['No Spam', 'Spam'])
plt.title('Matriz de Confusión - SVM Optimizado')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()


In [ ]:
# Comparación
print("\nCOMPARACIÓN DE MODELOS:")
print(f"{'Modelo':<20} {'Acc Train':<12} {'Acc Test':<12}")
print("-" * 44)
print(f"{'SVM Base':<20} {acc_train:.4f}       {acc_test:.4f}")
print(f"{'SVM Optimizado':<20} {acc_train_opt:.4f}       {acc_test_opt:.4f}")

Analisis de resultados:
Esto muestra que el modelo mejoró claramente en la detección de spam, sin aumentar los errores en la otra clase.

GUARDAR EL MODELO

In [ ]:
# Guardar vectorizador
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
print("Vectorizador guardado: vectorizer.pkl")

In [ ]:
# Guardar modelo optimizado
with open('svm_spam_detector.pkl', 'wb') as f:
    pickle.dump(svm_optimizado, f)
print("Modelo guardado: svm_spam_detector.pkl")